In [2]:
%pip install langchain

  Using cached langchain-0.0.306-py3-none-any.whl (1.8 MB)
  Using cached jsonpatch-1.33-py2.py3-none-any.whl (12 kB)
  Using cached langsmith-0.0.41-py3-none-any.whl (39 kB)
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 395.8/395.8 kB 8.3 MB/s eta 0:00:00a 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.4/49.4 kB 5.9 MB/s eta 0:00:00
  Using cached typing_inspect-0.9.0-py3-none-any.whl (8.8 kB)
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 8.8 MB/s eta 0:00:00a 0:00:01m
  Attempting uninstall: jsonpatch
    Found existing installation: jsonpatch 1.32
    Uninstalling jsonpatch-1.32:
      Successfully uninstalled jsonpatch-1.32
Note: you may need to restart the kernel to use updated packages.


In [5]:
%pip install openai

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.0/77.0 kB 1.9 MB/s eta 0:00:00a 0:00:01
Note: you may need to restart the kernel to use updated packages.


In [3]:
from langchain.schema import (
    AIMessage,
    HumanMessage,
    SystemMessage
)

In [67]:

from langchain.chat_models import ChatOpenAI

# create a chat model using openAI's gpt-4 model
chat = ChatOpenAI(model_name='gpt-4')  # , temperature=0

In [68]:
pre_prompt_old = """ACT as a corporate lawyer whose job is to update existing company policies according to new information.

Your task is to change the provided statement with the new regulation.

In the first step, identify and write which information is missing to update a statement.

Then, write which information is available to update the statement.

Finally, update the statement using as much information as possible and use placeholders for the parts that have missing information, such as [thing].

 In the end, ask for missing information, if needed. DO NOT ASK FOR UNNECESSARY INFORMATION that you can infer yourself.
 """

In [73]:
pre_prompt_old = """ACT as a corporate lawyer whose job is to update existing company policies according to new information.

Your task is to change the provided statement with the new regulation.

In the first step, identify and write which information is missing to update a statement.

Then, write which information is available to update the statement.

Finally, update the statement using as much information as possible and use placeholders for the parts that have missing information, such as [thing].

 In the end, ask for missing information, if needed. DO NOT ASK FOR UNNECESSARY INFORMATION that you can infer yourself.

 For example, if the statement is "The cat is the only animal in the house. We love the cat." and the new regulation is "You have to specify all family members and all animals in the house.", then the missing information is "The names and number of all family members in the house", the available information is "there is a cat in the house", and the updated statement is "The cat is the only animal in the house. We love both our family members and the cat."

 As another example, if the statement is "The company's address is 123 Main Street, New York, NY 10001. The company's phone number is 212-555-1234." and the new regulation is "You have to specify the company's address, phone number, and email address.", then the missing information is "the company's email address", the available information is "the company's address and phone number", and the updated statement is "The company's address is 123 Main Street, New York, NY 10001. The company's phone number is 212-555-1234. The company's email address is [email address]."
 """

In [89]:
pre_prompt = """ACT as a corporate lawyer whose job is to update existing company policies according to new information.

Your task is to change the provided statement with the new law.

In the first step, identify if any information is missing in the statement but is required by the new law, and if yes, write which information is missing.

Then, write which information is available to comply with the new law.

Finally, update the statement using available information and use placeholders for the parts that have missing information, such as [thing].

 In the end, ask for the missing information, if needed. DO NOT ASK FOR UNNECESSARY INFORMATION that you can infer yourself.

 For example, if the statement is "The cat is the only animal in the house. We love the cat." and the new law is "You have to specify all family members and all animals in the house.", then the missing information is "The names and number of all family members in the house", the available information is "there is a cat in the house", and the updated statement is "The cat is the only animal in the house. We love both our family members and the cat."

 As another example, if the statement is "The company's address is 123 Main Street, New York, NY 10001. The company's phone number is 212-555-1234." and the new law is "You have to specify the company's address, phone number, and email address.", then the missing information is "the company's email address", the available information is "the company's address and phone number", and the updated statement is "The company's address is 123 Main Street, New York, NY 10001. The company's phone number is 212-555-1234. The company's email address is [email address]."

 As another example, if the statement is "The users names, phones and emails are not stored in the database.", and the new law is "Neither phones, nor emails can be stored in the database", then there is no missing information, the available information is that "the users names are not stored in the database", and the updated statement is "The users names, phones and emails are not stored in the database."
 """


In [90]:
messages = [
    SystemMessage(content=pre_prompt),
    HumanMessage(content="""Statement: The cat is the only animal in the house. We love the cat.

previous regulation: [
You have to specify all animals in the house.
]

new regulation: [
You have to specify all family members and all animals in the house.
]"""
                 )
]
chat(messages)

AIMessage(content='Missing Information: The names and number of all family members in the house.\n\nAvailable Information: There is a cat in the house.\n\nUpdated Statement: The cat is the only animal in the house. We love both our family members and the cat. The family members in the house are [names and number of family members].\n\nPlease provide the names and number of all family members in the house.')

In [75]:
messages = [
    SystemMessage(content=pre_prompt),
    HumanMessage(content="""Statement: Mary likes Pete but Pete likes Anna. Mary is 5 years old.

previous regulation: [
 NONE
]

new regulation: [
you have to specify who each person, mentioned in the statement, likes
]

Updated statement:
"""
                 )
]
chat(messages)

AIMessage(content="Missing Information: Who Anna likes.\n\nAvailable Information: Mary likes Pete and Pete likes Anna. Mary's age is 5 years old.\n\nUpdated Statement: Mary likes Pete but Pete likes Anna. Anna likes [person]. Mary is 5 years old.\n\nPlease provide information on who Anna likes.")

In [91]:
messages = [
    SystemMessage(content=pre_prompt),
    HumanMessage(content="""Statement: Mary likes Pete but Pete likes Anna. Anna likes Mary. Mary is 5 years old.

Previous regulation: [
 NONE
]

New regulation: [
There has to be specified who each person likes.
]

"""
                 )
]
r = chat(messages)
r.content

'Missing Information: The information about who Anna likes is missing.\n\nAvailable Information: The information about who Mary and Pete like is available.\n\nUpdated Statement: Mary likes Pete but Pete likes Anna. Anna likes [person]. Mary is 5 years old.\n\nPlease provide the information about who Anna likes.'

In [79]:
messages = [

    HumanMessage(content=pre_prompt + """

    *** Statement ***
    Mary likes Pete but Pete likes Anna. Anna likes Mary. Mary is 5 years old.

*** Previous regulation ***
 NONE


*** New regulation ***
There has to be specified who each person likes.


"""
                 )
]
r = chat(messages)
r.content


"*** Missing Information ***\nThe information about who Anna likes is missing.\n\n*** Available Information ***\nThe available information is that Mary likes Pete, Pete likes Anna, and Anna likes Mary. Mary's age is also specified.\n\n*** Updated Statement ***\nMary likes Pete but Pete likes Anna. Anna likes Mary. Mary is 5 years old. Anna likes [person].\n\n*** Request for Missing Information ***\nPlease provide the information about who Anna likes."

In [64]:

chat([    HumanMessage(content='''ACT as a corporate lawyer whose job is to update existing company policies according to new information. Confirm by saying "yes". ''')])

AIMessage(content='Yes')

In [65]:
chat([    HumanMessage(content='''Who did I ask you to act like? ''')])

AIMessage(content="As an AI, I don't have the ability to recall past interactions. Could you please provide more context?")

In [70]:
from langchain.chat_models import ChatOpenAI

llm = ChatOpenAI()
llm.predict("Hello, world!")

'Hello! How can I assist you today?'

Tree of Thought

In [82]:
from langchain.prompts import PromptTemplate
from langchain.chains import LLMChain

In [83]:
template ="""
Step1 :

I have a problem related to {input}. Could you brainstorm three distinct solutions? Please consider a variety of factors such as {perfect_factors}
A:
"""

prompt = PromptTemplate(
    input_variables=["input","perfect_factors"],
    template = template
)

chain1 = LLMChain(
    llm=ChatOpenAI(temperature=0, model="gpt-4"),
    prompt=prompt,
    output_key="solutions"
)

template ="""
Step 2:

For each of the three proposed solutions, evaluate their potential. Consider their pros and cons, initial effort needed, implementation difficulty, potential challenges, and the expected outcomes. Assign a probability of success and a confidence level to each option based on these factors

{solutions}

A:"""

prompt = PromptTemplate(
    input_variables=["solutions"],
    template = template
)

chain2 = LLMChain(
    llm=ChatOpenAI(temperature=0, model="gpt-4"),
    prompt=prompt,
    output_key="review"
)

template ="""
Step 3:

For each solution, deepen the thought process. Generate potential scenarios, strategies for implementation, any necessary partnerships or resources, and how potential obstacles might be overcome. Also, consider any potential unexpected outcomes and how they might be handled.

{review}

A:"""

prompt = PromptTemplate(
    input_variables=["review"],
    template = template
)

chain3 = LLMChain(
    llm=ChatOpenAI(temperature=0, model="gpt-4"),
    prompt=prompt,
    output_key="deepen_thought_process"
)

template ="""
Step 4:

Based on the evaluations and scenarios, rank the solutions in order of promise. Provide a justification for each ranking and offer any final thoughts or considerations for each solution
{deepen_thought_process}

A:"""

prompt = PromptTemplate(
    input_variables=["deepen_thought_process"],
    template = template
)

chain4 = LLMChain(
    llm=ChatOpenAI(temperature=0, model="gpt-4"),
    prompt=prompt,
    output_key="ranked_solutions"
)

In [84]:
from langchain.chains import SequentialChain

overall_chain = SequentialChain(
    chains=[chain1, chain2, chain3, chain4],
    input_variables=["input", "perfect_factors"],
    output_variables=["ranked_solutions"],
    verbose=True
)

print(overall_chain({"input":"human colonization of Mars", "perfect_factors":"The distance between Earth and Mars is very large, making regular resupply difficult"}))



> Entering new SequentialChain chain...

> Finished chain.
{'input': 'human colonization of Mars', 'perfect_factors': 'The distance between Earth and Mars is very large, making regular resupply difficult', 'ranked_solutions': 'Building a Self-Sustaining Colony:\nPros: This solution would allow humans to live on Mars indefinitely, without needing constant resupply missions from Earth. It would also provide a stepping stone for further exploration of the solar system.\nCons: Building a self-sustaining colony would be a huge challenge, requiring advanced technology and a lot of resources. It would also require a long-term commitment, as the colony would need to be maintained and expanded over time.\nProbability of Success: 40%\nConfidence Level: Medium\n\nB: Mining Asteroids for Resources:\nPros: This solution could provide the resources needed to support a Martian colony, as well as potentially providing a new source of wealth. It could also help to reduce the cost of space exploration